In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from terminal_optimization import hydrogen_objects
from terminal_optimization import hydrogen_defaults
from terminal_optimization import hydrogen_system

# 0. Prepare inputs

In [ ]:
startyear = 2018
lifecycle = 10

## 0.1 Generate demand forecast scenario

In [ ]:
years = list(range(startyear, startyear+lifecycle))

demand = []
for year in years:
    if year < 2023:
        demand.append(750_000)
    else:
        demand.append(1_600_000)

scenario_data={'year': years, 'volume': demand}

# instantiate a commodity objects
# maize
hydrogen_defaults.maize_data['handysize_perc'] = 0
hydrogen_defaults.maize_data['handymax_perc'] = 100
hydrogen_defaults.maize_data['panamax_perc'] = 0
hydrogen_defaults.maize_data['handling_fee'] = 9.8 # example Wijnand
hydrogen_defaults.maize_data['historic_data'] = []
maize = hydrogen_objects.Commodity(**hydrogen_defaults.maize_data)
maize.scenario_data = pd.DataFrame(data=scenario_data)

# wheat
hydrogen_defaults.wheat_data['historic_data'] = []
hydrogen_defaults.wheat_data['scenario_data'] = []
wheat = hydrogen_objects.Commodity(**hydrogen_defaults.wheat_data)

# soybeans
hydrogen_defaults.soybean_data['historic_data'] = []
hydrogen_defaults.soybean_data['scenario_data'] = []
soybeans = hydrogen_objects.Commodity(**hydrogen_defaults.soybean_data)

# combine
demand = [maize, wheat, soybeans]

# plot scenarios 
if False:
    maize.plot_demand()
    wheat.plot_demand()
    soybeans.plot_demand()

## 0.2 Generate vessels

In [ ]:
# instantiate vessels
handysize = hydrogen_objects.Vessel(**hydrogen_defaults.handysize_data)
hydrogen_defaults.handymax_data['call_size'] = 50000
handymax = hydrogen_objects.Vessel(**hydrogen_defaults.handymax_data)
panamax = hydrogen_objects.Vessel(**hydrogen_defaults.panamax_data)


vessels = [handysize, handymax, panamax] 

# 1. Instatiate terminal system

In [ ]:
Terminal = hydrogen_system.System(startyear = startyear, 
                         lifecycle = lifecycle, 
                         elements = demand + vessels, 
                         operational_hours = 5840,   # example Wijnand
                         debug=True,
                         crane_type_defaults=hydrogen_defaults.mobile_crane_data, 
                         storage_type_defaults=hydrogen_defaults.silo_data)

# 2. Start simulation
Develop terminal given the throughput scenario and investment triggers implemented

In [ ]:
Terminal.simulate()

## Report all elements

In [ ]:
if False: 
    for element in Terminal.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__)

In [ ]:
Terminal.terminal_elements_plot()

In [ ]:
Terminal.terminal_capacity_plot()

In [ ]:
# add cash flow information for each of the Terminal elements
cash_flows, cash_flows_WACC_real = Terminal.add_cashflow_elements()

In [ ]:
Terminal.cashflow_plot(cash_flows)

In [ ]:
Terminal.cashflow_plot(cash_flows_WACC_real)

## SandBox

In [ ]:
Terminal.NPV()